# Skript um die diskretisierte DGL für die Massenerhaltung umzuformen


In [1]:
from sympy import symbols, expand, pprint, init_printing, latex, simplify, collect, Eq, IndexedBase
init_printing()  # nice pretty/LaTeX printing in notebooks

# Indizes & Parameter
i = symbols('i', integer=True)     # der Laufindex
delta_r = symbols('delta_r', positive=True)
r = IndexedBase('r')

# Felder (als diskrete Gitterwerte):
D_eff   = IndexedBase('D_eff')
rho_a = IndexedBase('rho_a')
w_f   = IndexedBase('w_f')
C   = symbols('C')                 # Skalar
w_sat_i = symbols('w_sat_i')
Alpha, Beta, Gamma = symbols('Alpha, Beta, Gamma')

eq = Eq(
    D_eff[i]*rho_a[i]*((w_f[i+1] - 2*w_f[i] + w_f[i-1])/(delta_r**2))
    + ( (D_eff[i]*rho_a[i])/r[i]
        + D_eff[i]*((rho_a[i+1] - rho_a[i-1])/(2*delta_r))
      ) * ((w_f[i+1] - w_f[i-1])/(2*delta_r)),
    C*rho_a[i]*(w_f[i] - w_sat_i)
)

print('Eingegeben Gleichung:')
pprint(eq)

# Alles auf eine Seite bringen -> Ausdruck
expr = expand(eq.lhs - eq.rhs)

# Nach w_m, w_i, w_n gruppieren
col = collect(expr, [w_f[i+1], w_f[i], w_f[i-1]], evaluate=False)
alpha = simplify(col.get(w_f[i+1], 0))
beta = simplify(col.get(w_f[i],   0))
gamma = simplify(col.get(w_f[i-1], 0))
const = simplify(col.get(1, 0))     # Restterm ohne w_m, w_i, w_n

# Rekonstruktion der linken Seite in gewünschter Form
lhs_grouped = w_f[i+1]*alpha + w_f[i]*beta + w_f[i-1]*gamma + const

# Als Gleichung (== 0) oder wieder nach rechts umstellen:
eq_grouped_zero = Eq(lhs_grouped, 0)
eq_grouped      = Eq(w_f[i+1]*alpha + w_f[i]*beta + w_f[i-1]*gamma, -const)

print('\nGruppiert für Matrix Notation:\n')
pprint(eq_grouped)

print('\nFaktoren:\n')
pprint(Eq(Alpha, alpha))
pprint(Eq(Beta, beta))
pprint(Eq(Gamma, gamma))

print('\nLaTeX:\n')
print(latex(eq_grouped))

Eingegeben Gleichung:
⎛D_eff[i]⋅rho_a[i]   (rho_a[i + 1] - rho_a[i - 1])⋅D_eff[i]⎞                   ↪
⎜───────────────── + ──────────────────────────────────────⎟⋅(w_f[i + 1] - w_f ↪
⎝      r[i]                           2⋅δᵣ                 ⎠                   ↪
────────────────────────────────────────────────────────────────────────────── ↪
                                         2⋅δᵣ                                  ↪
                                                                               ↪

↪                                                                              ↪
↪ [i - 1])                                                                     ↪
↪            (w_f[i + 1] + w_f[i - 1] - 2⋅w_f[i])⋅D_eff[i]⋅rho_a[i]            ↪
↪ ──────── + ────────────────────────────────────────────────────── = C⋅(-wₛₐₜ ↪
↪                                       2                                      ↪
↪                                     δᵣ                                       ↪

↪   